# NH MU69 Rings Planning

## Initialize python

In [102]:
import astropy.units as u
import astropy.constants as c
import math
import astropy
import numpy as np
import matplotlib.pyplot as plt
import pymiecoated # Mie scattering library
import spiceypy as sp
from astropy.table import Table

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
                                    # Ignore warnings caused by pymiecoated's numpy usage 

import matplotlib                   # Set latex to use *not* CMR.
matplotlib.rcParams['mathtext.fontset'] = 'stixsans'

r2d = 360 / (2*math.pi)             # Radians to degrees
d2r = 1/r2d

file_kernel = '/Users/throop/gv/dev/gv_kernels_new_horizons.txt'

## Set up some physical constants for NH and the encounter

In [178]:
a_pluto_ca   = 33*u.au             # NH heliocentric distance at Pluto flyby
a_mu69_ca    = 43*u.au             # NH heliocentric distance at MU69 flyby

a_ring_phoebe_inner = (50*t['radius'][index_saturn])
a_ring_phoebe_outer = (300*t['radius'][index_saturn])

rho_pluto       = 2.5 * u.gram/u.cm**3
rho_mu69        = 2.5 * u.gram/u.cm**3

a_hydra      = 65000 * u.km        # Hydra orbital distance around Pluto

fov_mvic     = 5.7 * d2r           # MVIC FOV, in radians
fov_lorri    = 0.3 * d2r           # LORRI FOV width, in radians 

sp.furnsh(file_kernel)

## Calculate the Hill sphere for a few bodies

$$r_H = a \sqrt[3]{m\over {3 M}}$$

In [200]:
# Set up an astropy table with data for a few different planets

name = np.array(['Pluto', 'MU69', 'Saturn', 'Chariklo'])
rho  = np.array([2.5,     2.5,     1,        2.5])*u.gram/u.cm**3
r    = np.array([1187, 16.5, 58232, 200])*u.km
a    = np.array([33, 43, 10, 15.7])*u.au

# Create the table

t = Table([name, rho, r, a], names=['Name', 'rho', 'radius', 'a'])

# Now do some math on the table. Note that for weird reasons, table columns' units get messed up if
# we just directly multiply them. We need to use .quantity to do it properly. This is documented
# but weird. See bottom of http://docs.astropy.org/en/stable/table/access_table.html

t['mass'] = (4/3. * math.pi * (t['radius'].quantity)**3 * t['rho'].quantity).to('kg')
t['a_hill']  = (a * (t['mass'].quantity/c.M_sun/3)**(1/3)).to('km')

# Add some more columns. 
t['a_hill / a_hydra'] = t['a_hill'].quantity / a_hydra
t['a_hill / r_<body>'] = t['a_hill'].quantity / t['radius']

# Add some shortcuts to access individual rows. 
# Astropy does not allow e.g., t['Radius']['Saturn'] -- have to do t['Radius'][index_saturn]

index_pluto = t['Name'] == 'Pluto'
index_mu69  = t['Name'] == 'MU69'
index_saturn= t['Name'] == 'Saturn'
index_chariklo=t['Name'] == 'Chariklo'

In [201]:
t

Name,rho,radius,a,mass,a_hill,a_hill / a_hydra,a_hill / r_<body>
,g / cm3,km,AU,kg,km,,
str8,float64,float64,float64,float64,float64,float64,float64
Pluto,2.5,1187.0,33.0,1.75138156829e+22,7068169.99638,108.741076867,5954.65037606
MU69,2.5,16.5,43.0,4.70414229967e+16,128024.983085,1.96961512439,7759.08988395
Saturn,1.0,58232.0,10.0,8.27129915151e+26,77420748.216,1191.08843409,1329.5223969
Chariklo,2.5,200.0,15.7,8.37758040957e+19,566594.005479,8.71683085353,2832.9700274


In [195]:
print("rh_pluto = {:,.1f} = {:,.1f} r_pluto = {:,.1f} a_hydra".format(rh_pluto.to('km'), 
                                                                     (rh_pluto/r_pluto).to(''),
                                                                     (rh_pluto/a_hydra).to('')))
print("rh_mu69  = {:,.1f} = {:.1f} r_mu69 = {:,.1f} a_hydra ".format (rh_mu69.to('km'),  
                                                                     (rh_mu69/r_mu69).to(''),
                                                                     (rh_mu69/a_hydra).to('')))

rh_pluto = 7,068,170.0 km = 5,954.7 r_pluto = 108.7 a_hydra
rh_mu69  = 128,025.0 km = 7759.1 r_mu69 = 2.0 a_hydra 


In [197]:
print(t['Name'], t['a_hill'].quantity/a_hydra)

  Name  
--------
   Pluto
    MU69
  Saturn
Chariklo [  108.74107687     1.96961512  1227.25144329    13.88030391]


Oh wow. So, at Pluto it made sense to search at P+100 days, because that was basically the Hill radius. But at MU69, the Hill radius is 50x smaller, so all the searching must be done much closer in.

At what point does MVIC see out to the Hill radius on each side of MU69?

In [98]:
### Calc NH velocity. This kernel doesn't have MU69 in it, so just use Pluto flyby. Velocities are about the same.

utc = '1 Jan 2016'
et = sp.utc2et(utc)
(st,lt) = sp.spkezr('Pluto', et, 'J2000', 'LT+S', 'New Horizons')

In [93]:
v_nh = sp.vnorm(st[3:6])*u.km/u.second

In [171]:
t_mu69_rh = (rh_mu69 / (fov_mvic/2) / v_nh)
print("At K+{:.1f}, MVIC FOV halfwidth is 1 R_Hill(MU69) in each direction.".format(t_mu69_rh.to('day')))

At K+2.2 d, MVIC FOV halfwidth is 1 R_Hill(MU69) in each direction.


In [172]:
# Where is the Phoebe ring, in hill spheres?

In [175]:
index_pluto

array([ True, False, False, False], dtype=bool)

In [187]:
print(t['a_h'][index_saturn].quantity / t['radius'][index_saturn].quantity)

[ 1329.5223969]


In [189]:
# Calculate Phoebe ring in Saturn hill radii

print(a_ring_phoebe_outer / t['a_hill'][index_saturn].quantity)

[ 0.22564494]
